In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Model
from azureml.core.conda_dependencies import CondaDependencies

# working with script argument 

In [2]:
ws = Workspace.from_config()

# Here we are using the environment created in previous notebook. 
# We can retrive the environment because it was registered in the workspace. 
myenv = Environment.get(workspace=ws, 
                        name='environemnt-002')

In [3]:
# We train a random forest model in the script. 
# We can provide the hyperparameter of the model as an argument in the 
# script_config object which is parsed in the script. 
script_config = ScriptRunConfig(source_directory='.', 
                                script='script_3.py', 
                                arguments=['--num_tree', 5], 
                                environment=myenv)

In [4]:
# This part of the code is same as before. 
experiment = Experiment(workspace=ws, name='azureml-demo-exp')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)

RunId: azureml-demo-exp_1633633631_abe736e0
Web View: https://ml.azure.com/runs/azureml-demo-exp_1633633631_abe736e0?wsid=/subscriptions/54245888-2ffe-41fa-b080-67a29997b41c/resourcegroups/rg-dataservices-sandbox-01/workspaces/ds_dev_01&tid=4ef6e02a-f252-4618-a1dc-03bd2f93157d

Streaming azureml-logs/70_driver_log.txt

[2021-10-07T19:07:14.425224] Entering context manager injector.
[2021-10-07T19:07:15.214391] context_manager_injector.py Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError'], invocation=['script_3.py', '--num_tree', '5'])
Script type = None
[2021-10-07T19:07:15.219378] Entering Run History Context Manager.
[2021-10-07T19:07:16.383198] Current directory: C:\Users\rkoirala\AppData\Local\Temp\azureml_runs\azureml-demo-exp_1633633631_abe736e0
[2021-10-07T19:07:16.383198] Preparing to call script [script_3.py] with arguments:['--num_tree', '5'

{'runId': 'azureml-demo-exp_1633633631_abe736e0',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-10-07T19:07:13.906208Z',
 'endTimeUtc': '2021-10-07T19:07:43.892658Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '0da8c1d8-e105-4508-95b7-bee800dc410d',
  'azureml.git.repository_uri': 'https://github.com/roshankoirala/MLOps.git',
  'mlflow.source.git.repoURL': 'https://github.com/roshankoirala/MLOps.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'cf58f46e4a0bde9e6012ad78d77de1497aa012ce',
  'mlflow.source.git.commit': 'cf58f46e4a0bde9e6012ad78d77de1497aa012ce',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'script_3.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--num_tree', '5'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'loca

# Downloading model files 

In [5]:
# We can see the files logged during the experiment. 
# First three files are produced by default in azure. 
# The last file is the serialized model we logged for this run 
# through the script. 
for file in run.get_file_names():
    print(file)

azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/16136_azureml.log
outputs/model.pkl


In [6]:
# We can download the model from the cloud to the local computer. 
run.download_file(name='outputs/model.pkl', output_file_path='model.pkl')

# Register a model 

In addition to `Experiments` the azure ml has another very important class called `Models` where we can register the models which we can later use for the production. Here we present two ways to register a model. 
- We may train the model locally. Then save the model locally as a `.pkl` or similar file. We can upload the register the model in the cloud. 
- We may run the experiment and have model as logged file from the experiment already present in the cloud. We can register the logged model directly as well. 

In [7]:
# From local file 
model = Model.register(workspace=ws, 
                       model_name='demo_iris_model', 
                       model_path='model.pkl',  # local path 
                       description='Classification model for iris data', 
                       tags={'model-class':'random-forest', 
                             'preprocessing':'zero-feature-engineering'}, 
                       model_framework=Model.Framework.SCIKITLEARN)

Registering model demo_iris_model


In [8]:
# from cloud output 
run.register_model(model_name='demo_iris_model', 
                   model_path='outputs/model.pkl', 
                   description='Classification model for iris data', 
                   tags={'model-class':'random-forest', 
                             'preprocessing':'zero-feature-engineering'}, 
                   model_framework=Model.Framework.SCIKITLEARN)

Model(workspace=Workspace.create(name='ds_dev_01', subscription_id='54245888-2ffe-41fa-b080-67a29997b41c', resource_group='rg-dataservices-sandbox-01'), name=demo_iris_model, id=demo_iris_model:2, version=2, tags={'model-class': 'random-forest', 'preprocessing': 'zero-feature-engineering'}, properties={})

In [9]:
# Model with the same name are saved as a separate version of the same model by default. 
# We can list the name and version of the model in the following way. 
for model in Model.list(ws):
    print(model.name, 'Version:', model.version)

demo_iris_model Version: 2
demo_iris_model Version: 1
amlstudio-test-deploy-designer Version: 4
amlstudio-test-deploy-designer Version: 3
amlstudio-test-deploy-designer Version: 2
amlstudio-test-deploy-designer Version: 1
lgbm_model_Aug_2 Version: 1
lgbm_model_Jul-09-2021 Version: 1
RFGridSearchCV Version: 1
